In [4]:
import datetime
import json
import os
import matplotlib.pyplot as plt
import psutil
import seaborn as sns
import numpy as np
import networkx as nx
import pandas as pd

sns.set_theme()


def trim_agent(agent):
    if agent.startswith("/"):
        agent = agent[1:]
    if agent.startswith("go-ipfs"):
        return "go-ipfs"
    elif agent.startswith("hydra-booster"):
        return "hydra-booster"
    elif agent.startswith("storm"):
        return "storm"
    elif agent.startswith("ioi"):
        return "ioi"
    elif agent.startswith("n.a."):
        return "n.a."
    else:
        return "others"


def get_millis(duration):
    if duration.endswith("µs"):
        return float(duration[:-2]) / 1000
    elif duration.endswith("ms"):
        return float(duration[:-2])
    elif duration.endswith("s"):
        if "m" in duration:
            return int(duration[0]) * 60 * 1000 + get_millis(duration[2:])
        return float(duration[:-2]) * 1000


def millis_interval(start_time, end_time):
    """start and end are datetime instances"""
    diff = end_time - start_time
    millis = diff.days * 24 * 60 * 60 * 1000
    millis += diff.seconds * 1000
    millis += diff.microseconds / 1000
    return millis

class Node:
 
    def __init__(self, address):
        self.address = address
        self.connect = set()


In [54]:
hydra_list = [
    "...",
]
hydra_set = set(hydra_list)
def is_hydra(pid):
    return pid in hydra_set

In [5]:
def log_num_to_region(i):
    log_no_to_region = {
        0: "me_south_1",
        1: "ap_southeast_2",
        2: "af_south_1",
        3: "us_west_1",
        4: "eu_central_1",
        5: "sa_east_1",
    }
    return log_no_to_region[i]

In [88]:
def get_dht_walk_graphs(logs):
    # Results for pvd
    pvd_latencies = []
    pvd_dht_walk_latencies = []
    pvd_put_latencies = []
    pvd_put_latencies_succeed = []
    pvd_put_latencies_failed = []
    pvd_agents = dict()
    pvd_dht_walk_agents = dict()
    pvd_put_agents = dict()
    pvd_put_agents_succeed = dict()
    pvd_put_agents_failed = dict()
    # Results for ret
    ret_latencies = []
    ret_dht_walk_latencies = []
    ret_get_latencies = []
    ret_agents = dict()
    ret_dht_walk_agents = dict()
    ret_get_agents = dict()
    # Error messages.
    errs = []
    # Error getting provider record after a successful put
    agents_1 = dict()
    errs_1 = []
    # Get empty provider record after a successful put
    agents_2 = dict()
    # CID -> (Ret DHT Walk Graph, Destination)
    graphs = {}
    for file in logs:
        pvd_on = False
        pvd_start = None
        pvd_search = None
        ret_on = False
        dht_walk_on = False
        ret_start = None
        ret_search = None
        # DHT Walk Graph
        nodes = {}
        provider_addr = None
        known_nodes = set()
        if not os.path.isfile(file):
            break
        for line in reversed(list(open(file))):
            data = json.loads(line)
            data["line"] = data["line"].split("Z: ")[1] if len(data["line"].split("Z: ")) > 1 else data["line"] 
            if data["line"].startswith("Start providing"):
                pvd_start = datetime.datetime.strptime(
                    data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                pvd_on = True
            elif data["line"].startswith("Finish providing"):
                pvd_on = False
                # just to skip a bug, not correct
                if pvd_search is None:
                    continue
                ##################################
                end = datetime.datetime.strptime(
                    data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                pvd_put_latencies.append(millis_interval(pvd_search, end))
                pvd_latencies.append(millis_interval(pvd_start, end))
                
            elif data["line"].startswith("Start retrieving"):
                ret_start = datetime.datetime.strptime(
                    data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                ret_on = True
                dht_walk_on = True
            elif data["line"].startswith("Done retrieving"):
                end = datetime.datetime.strptime(
                    data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                # Just to skip a bug, not correct
                if ret_search is None:
                    ret_search = end
                if ret_start is None or end is None:
                    print(data["line"].split(" ")[4])
                ########################
                ret_get_latencies.append(millis_interval(ret_search, end))
                ret_latencies.append(millis_interval(ret_start, end))
                cid = data["line"].split(" ")[4]
                graphs[file + "." + cid] = (nodes, provider_addr)
                nodes = {}
                provider_addr = None
                known_nodes = set()
                ret_on = False
                dht_walk_on = False
            else:
                if pvd_on:
                    agent = trim_agent(data["line"].split("(")[-1].split(")")[0])
                    if data["line"].startswith("Getting closest peers for cid"):
                        pvd_agents[agent] = pvd_agents.get(agent, 0) + 1
                        pvd_dht_walk_agents[agent] = pvd_dht_walk_agents.get(
                            agent, 0) + 1
                    elif data["line"].startswith("Succeed in putting provider record"):
                        pvd_agents[agent] = pvd_agents.get(agent, 0) + 1
                        pvd_put_agents[agent] = pvd_put_agents.get(agent, 0) + 1
                        pvd_put_agents_succeed[agent] = pvd_put_agents_succeed.get(
                            agent, 0) + 1
                        latency = data["line"].split(":")[-1][1:]
                        pvd_put_latencies_succeed.append(get_millis(latency))
                    elif data["line"].startswith("Error putting provider record"):
                        pvd_agents[agent] = pvd_agents.get(agent, 0) + 1
                        pvd_put_agents[agent] = pvd_put_agents.get(agent, 0) + 1
                        pvd_put_agents_failed[agent] = pvd_put_agents_failed.get(
                            agent, 0) + 1
                        latency = data["line"].split(":")[-1][1:]
                        pvd_put_latencies_failed.append(get_millis(latency))
                        if agent == "hydra-booster":
                            errs.append(data["line"].split(")")[
                                        1].split("time taken")[0])
                    elif data["line"].startswith("In total"):
                        pvd_search = datetime.datetime.strptime(
                            data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                        pvd_dht_walk_latencies.append(
                            millis_interval(pvd_start, pvd_search))
                    elif data["line"].startswith("Error getting provider record for cid"):
                        agents_1[agent] = agents_1.get(agent, 0) + 1
                        errs_1.append(data["line"].split(
                            "after a successful put")[1])
                    elif data["line"].startswith("Got 0 provider records back from"):
                        agents_2[agent] = agents_2.get(agent, 0) + 1
                if ret_on:
                    agent = trim_agent(data["line"].split("(")[-1].split(")")[0])
                    if data["line"].startswith("Getting providers for cid"):
                        ret_agents[agent] = ret_agents.get(agent, 0) + 1
                        ret_dht_walk_agents[agent] = ret_dht_walk_agents.get(
                            agent, 0) + 1
                        if dht_walk_on:
                            pid = data["line"].split(" ")[6].split("(")[0]
                            known_nodes.add(pid)
                            node = Node(pid)
                            nodes[pid] = node
                    elif data["line"].startswith("Got") and data["line"].split(" ")[2] == "closest":
                        if dht_walk_on:
                            # print(from_id + ": " + str(known_nodes))
                            from_id = data["line"].split(" ")[8].split("(")[0]
                            
                            peers = data["line"].split(": ")[1].split(" ")
                            for peer in peers:
                                if peer not in known_nodes:
                                    nodes[from_id].connect.add(peer)
                                    known_nodes.add(peer)
                            # print(from_id + "\n" + str(nodes[from_id].connect))
                            # print(from_id + "\n" + str(peers))

                    elif data["line"].startswith("Found provider"):
                        ret_agents[agent] = ret_agents.get(agent, 0) + 1
                        ret_get_agents[agent] = ret_get_agents.get(agent, 0) + 1
                        if dht_walk_on:
                            splitted = data["line"].split(" ")
                            pid = splitted[2]
                            from_id = splitted[7].split("(")[0]
                            nodes[from_id].connect.add(pid)
                            node = Node(pid)
                            nodes[pid] = node
                            provider_addr = pid
                            dht_walk_on = False

                    elif data["line"].startswith("Connected to provider"):
                        ret_search = datetime.datetime.strptime(
                            data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                        ret_dht_walk_latencies.append(
                            millis_interval(ret_start, ret_search))
                    pass
                pass
    return graphs


In [7]:
def build_graph(nodes):
    if nodes == {}:
        return None
    G = nx.DiGraph() 
    # nodes = graphs["Qmat5rRqrC8uNNMp6fgADwb2w274fns3co5QR2sSEYRZKB"][0]
    for name, node in nodes.items():
        G.add_node(name)
        for to_node in node.connect:
            if to_node in nodes.keys():
                G.add_edge(name, to_node)
    G.add_node("Local")
    for node in G.nodes:
        if node != "Local" and G.in_degree(node) == 0:
            G.add_edge("Local", node)
    return G


In [8]:
def merge_graph(graphs):
    path_length = {}
    G = nx.DiGraph()
    # error_count = 0
    for name, (nodes, provider) in graphs.items():
        g = build_graph(nodes)
        if g is None or provider is None:
            # error_count += 1
            continue
        path = nx.dijkstra_path(g, "Local", provider)
        path_length[name] = len(path)
        for i in range(len(path) - 1):
            n1 = path[i]
            n2 = path[i+1]
            if (n1, n2) in G.edges:
                G[n1][n2]["weight"] = G[n1][n2]["weight"] + 1
            else:
                G.add_edge(n1, n2, weight=1)
    return G, path_length


In [9]:
def count_path_length(path_length):
    df = pd.DataFrame({"cid": path_length.keys(), "path_length": path_length.values()})
    count = df.groupby("path_length").count()
    count["cid"] = list(map(lambda x: x / sum(count["cid"]), count["cid"]))
    return count

In [10]:
def compare_path_length(with_hydra, withou_hydra):
    c1 = count_path_length(with_hydra)
    c2 = count_path_length(withou_hydra)
    c1 = c1.rename({"cid": "with_hydra"}, axis=1)
    c2 = c2.rename({"cid": "without_hydra"}, axis=1)
    return c1.join(c2)

In [11]:
def compare_hot_nodes(with_hydra, without_hydra):
       graphs1 = with_hydra
       graphs2 = without_hydra
       G1, path_length1 = merge_graph(graphs1)
       G2, path_length2 = merge_graph(graphs2)
       degrees1 = [(node, G1.in_degree(node, weight="weight")) for node in G1.nodes]
       s1 = sorted(degrees1, key=lambda x: x[1], reverse=True)

       degrees2 = [(node, G2.in_degree(node, weight="weight")) for node in G2.nodes]
       s2 = sorted(degrees2, key=lambda x: x[1], reverse=True)

       s1 = s1[5:35]
       s2 = s2[5:35]

       dic = {"with_hydra": [count / len(graphs1) for count in map(lambda x: x[1], s1)],
              "without_hydra": [count / len(graphs2) for count in map(lambda x: x[1], s2)]}

       df = pd.DataFrame(dic)
       return df

In [ ]:
for i in range(6):
    f1 = "./2022-10-14-data/" + str(i) + ".log"
    f2 = "./2022-11-07-data/" + str(i) + ".log"

    graphs1 = get_dht_walk_graphs([f1])
    G1, path_length1 = merge_graph(graphs1)

    graphs2 = get_dht_walk_graphs([f2])
    G2, path_length2 = merge_graph(graphs2)

    fig = compare_path_length(path_length1, path_length2).plot(kind="bar", ylabel="Retrieve Count (%)", title=log_num_to_region(i))
    plt.savefig("./figs/path_length/" + log_num_to_region(i) + ".pdf")
    
    fig = compare_hot_nodes(graphs1, graphs2).plot(kind="line", xlabel="Top 30 Queried Nodes", ylabel="Query Count (Normalized)", title=log_num_to_region(i))
    plt.savefig("./figs/hot_nodes/" + log_num_to_region(i) + ".pdf")

In [87]:
def get_nodes_latency_from_logs(logs):

    # node -> (cid -> latency)
    latencies = {}
    # node -> latency
    tmp_latencies = {}

    # Results for pvd
    pvd_latencies = []
    pvd_dht_walk_latencies = []
    pvd_put_latencies = []
    pvd_put_latencies_succeed = []
    pvd_put_latencies_failed = []
    pvd_agents = dict()
    pvd_dht_walk_agents = dict()
    pvd_put_agents = dict()
    pvd_put_agents_succeed = dict()
    pvd_put_agents_failed = dict()
    # Results for ret
    ret_latencies = []
    ret_dht_walk_latencies = []
    ret_get_latencies = []
    ret_agents = dict()
    ret_dht_walk_agents = dict()
    ret_get_agents = dict()
    # Error messages.
    errs = []
    # Error getting provider record after a successful put
    agents_1 = dict()
    errs_1 = []
    # Get empty provider record after a successful put
    agents_2 = dict()
    # CID -> (Ret DHT Walk Graph, Destination)
    graphs = {}
    for file in logs:
        pvd_on = False
        pvd_start = None
        pvd_search = None
        ret_on = False
        dht_walk_on = False
        ret_start = None
        ret_search = None
        # DHT Walk Graph
        nodes = {}
        provider_addr = None
        known_nodes = set()
        if not os.path.isfile(file):
            break
        for line in reversed(list(open(file))):
            data = json.loads(line)
            tstamp = data["line"].split("Z: ")[0] if len(data["line"].split("Z: ")) > 1 else None
            tstamp = datetime.datetime.strptime(tstamp[:-3], "%Y-%m-%dT%H:%M:%S.%f") if tstamp is not None else None
            data["line"] = data["line"].split("Z: ")[1] if len(data["line"].split("Z: ")) > 1 else data["line"] 
            if data["line"].startswith("Start providing"):
                pvd_start = datetime.datetime.strptime(
                    data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                pvd_on = True
            elif data["line"].startswith("Finish providing"):
                pvd_on = False
                # just to skip a bug, not correct 
                if pvd_search is None:
                    continue
                ###############################
                end = datetime.datetime.strptime(
                    data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                pvd_put_latencies.append(millis_interval(pvd_search, end))
                pvd_latencies.append(millis_interval(pvd_start, end))
                
            elif data["line"].startswith("Start retrieving"):
                ret_start = datetime.datetime.strptime(
                    data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                ret_on = True
                dht_walk_on = True
            elif data["line"].startswith("Done retrieving"):
                end = datetime.datetime.strptime(
                    data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                # Just to skip a bug, not correct
                if ret_search is None:
                    ret_search = end
                if ret_start is None or end is None:
                    print(data["line"].split(" ")[4])
                ########################
                ret_get_latencies.append(millis_interval(ret_search, end))
                ret_latencies.append(millis_interval(ret_start, end))
                cid = data["line"].split(" ")[4]
                graphs[file + "." + cid] = (nodes, provider_addr)
                nodes = {}
                provider_addr = None
                known_nodes = set()
                ret_on = False
                dht_walk_on = False
            else:
                if pvd_on:
                    agent = trim_agent(data["line"].split("(")[-1].split(")")[0])
                    if data["line"].startswith("Getting closest peers for cid"):
                        pvd_agents[agent] = pvd_agents.get(agent, 0) + 1
                        pvd_dht_walk_agents[agent] = pvd_dht_walk_agents.get(
                            agent, 0) + 1
                    elif data["line"].startswith("Succeed in putting provider record"):
                        pvd_agents[agent] = pvd_agents.get(agent, 0) + 1
                        pvd_put_agents[agent] = pvd_put_agents.get(agent, 0) + 1
                        pvd_put_agents_succeed[agent] = pvd_put_agents_succeed.get(
                            agent, 0) + 1
                        latency = data["line"].split(":")[-1][1:]
                        pvd_put_latencies_succeed.append(get_millis(latency))
                    elif data["line"].startswith("Error putting provider record"):
                        pvd_agents[agent] = pvd_agents.get(agent, 0) + 1
                        pvd_put_agents[agent] = pvd_put_agents.get(agent, 0) + 1
                        pvd_put_agents_failed[agent] = pvd_put_agents_failed.get(
                            agent, 0) + 1
                        latency = data["line"].split(":")[-1][1:]
                        pvd_put_latencies_failed.append(get_millis(latency))
                        if agent == "hydra-booster":
                            errs.append(data["line"].split(")")[
                                        1].split("time taken")[0])
                    elif data["line"].startswith("In total"):
                        pvd_search = datetime.datetime.strptime(
                            data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                        pvd_dht_walk_latencies.append(
                            millis_interval(pvd_start, pvd_search))
                    elif data["line"].startswith("Error getting provider record for cid"):
                        agents_1[agent] = agents_1.get(agent, 0) + 1
                        errs_1.append(data["line"].split(
                            "after a successful put")[1])
                    elif data["line"].startswith("Got 0 provider records back from"):
                        agents_2[agent] = agents_2.get(agent, 0) + 1
                if ret_on:
                    agent = trim_agent(data["line"].split("(")[-1].split(")")[0])
                    if data["line"].startswith("Getting providers for cid"):
                        ret_agents[agent] = ret_agents.get(agent, 0) + 1
                        ret_dht_walk_agents[agent] = ret_dht_walk_agents.get(
                            agent, 0) + 1
                        if dht_walk_on:
                            pid = data["line"].split(" ")[6].split("(")[0]
                            known_nodes.add(pid)
                            node = Node(pid)
                            nodes[pid] = node
                            
                            tmp_latencies[pid] = tstamp

                    elif data["line"].startswith("Got") and data["line"].split(" ")[2] == "closest":
                        if dht_walk_on:
                            # print(from_id + ": " + str(known_nodes))
                            from_id = data["line"].split(" ")[8].split("(")[0]
                            
                            peers = data["line"].split(": ")[1].split(" ")
                            for peer in peers:
                                if peer not in known_nodes:
                                    nodes[from_id].connect.add(peer)
                                    known_nodes.add(peer)
                            # print(from_id + "\n" + str(nodes[from_id].connect))
                            # print(from_id + "\n" + str(peers))

                    elif data["line"].startswith("Found provider"):
                        ret_agents[agent] = ret_agents.get(agent, 0) + 1
                        ret_get_agents[agent] = ret_get_agents.get(agent, 0) + 1
                        if dht_walk_on:
                            splitted = data["line"].split(" ")
                            pid = splitted[2]
                            from_id = splitted[7].split("(")[0]
                            nodes[from_id].connect.add(pid)
                            node = Node(pid)
                            nodes[pid] = node
                            provider_addr = pid
                            dht_walk_on = False
                    elif data["line"].startswith("Found 0") or data["line"].startswith("Found 1"):
                        cid = data["line"].split(" ")[6]
                        pid = data["line"].split(" ")[8].split("(")[0]
                        if dht_walk_on:
                            # tstamp = datetime.datetime.strptime(data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                            latency = millis_interval(tmp_latencies[pid], tstamp)
                            if pid not in latencies.keys():
                                latencies[pid]= {}
                            latencies[pid][cid] = latency
                    elif data["line"].startswith("Connected to provider"):
                        ret_search = datetime.datetime.strptime(
                            data["timestamp"][:-9], "%Y-%m-%dT%H:%M:%S.%f")
                        ret_dht_walk_latencies.append(
                            millis_interval(ret_start, ret_search))
                    pass
                pass
    return latencies


In [38]:
def get_nodes_avg_latency(la):
    return [(pid, sum(la[pid].values()) / len(la[pid])) for pid in la.keys()]

In [66]:
def get_degree_latency_df(logs):
    graphs1 = get_dht_walk_graphs(logs)
    G1, path_length1 = merge_graph(graphs1)
    degrees1 = [(node, G1.in_degree(node, weight="weight")) for node in G1.nodes]
    degrees1 = [(p[0], p[1] / len(graphs1)) for p in degrees1]

    la = get_nodes_latency_from_logs(logs)
    avg_la = get_nodes_avg_latency(la)

    df1 = pd.DataFrame({"pid":[p[0] for p in degrees1], "degree":[p[1] for p in degrees1]})
    df2 = pd.DataFrame({"pid":[p[0] for p in avg_la], "latency":[p[1] for p in avg_la]})
    df1 = df1.set_index("pid").sort_index()
    df2 = df2.set_index("pid").sort_index()
    df = df1.join(df2, how="inner")
    df["hydra"] = list(map(is_hydra, df.index))
    
    
    return df




In [ ]:
for i in range(6):
    f1 = "./2022-10-26-data/" + str(i) + ".log"
    fig = get_degree_latency_scatter([f1])
    fig.set_title(log_num_to_region(i))
    plt.savefig("./figs/latency-degree/" + log_num_to_region(i) + ".pdf")
